# **Initial setup**

Install Bambu and required packages:

In [ ]:
!echo "deb http://ppa.launchpad.net/git-core/ppa/ubuntu $(cat /etc/os-release | grep UBUNTU_CODENAME | sed 's/.*=//g') main" >> /etc/apt/sources.list.d/git-core.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-keys A1715D88E1DF1F24
!apt-get update
!apt-get install -y --no-install-recommends build-essential ca-certificates gcc-multilib git libtinfo5 verilator wget
!wget https://release.bambuhls.eu/appimage/bambu-latest-dev.AppImage
!chmod +x bambu-*.AppImage
!ln -sf $PWD/bambu-*.AppImage /bin/bambu
!ln -sf $PWD/bambu-*.AppImage /bin/spider
!git clone --depth 1 --filter=blob:none --branch dev/panda --sparse https://github.com/ferrandi/PandA-bambu.git
%cd PandA-bambu
!git sparse-checkout set documentation/bambu101
%cd ..
!mv PandA-bambu/documentation/bambu101/* .

## Input description requirements
The HLS tool requires an input description of the function to be translated into hardware: any kind of function is synthesizable provided that it is self-contained (pre-compiled external libraries are not allowed) and it is not a template function.

Bambu HLS allows the use of data structures, arrays, pointer references and multiple indirection, memcpy, memcmp, and memset functions. Dynamic memory allocation is strongly discouraged since it is not optimized nor granted to be free from runtime errors while it is still possible to perform the synthesis when malloc and free are used within the input description.

It is also possible to exploit STL containers through the Embedded Template Library (ETL) which provides statically allocated containers (vector, deque, ...) with an STL-compliant API.

C++ standard up to C++20 is supported depending on the selected front-end compiler.

## AES Encryption Application
The AES algorithm is a widely used encryption algorithm which is often provided as a hardware module. In the following, it shown how to generate such a hardware accelerated version of the AES encrypt function starting from a C software description.
A basic testbench and an example application are provided along with the AES encryption kernel C description.

# Generating the accelerator
To generate a basic version of the hardware design it is necessary to specify just a few options:
 - Input file: _aes.c_
 - Top function name: _--top-fname=aes256_encrypt_ecb_
 - Target device: _--device-name=xc7z020-1clg484-VVD_
 - Target clock period: _--clock-period=5_

In [ ]:
%cd /content/kernel_offload/aes
!bambu aes.c -I../common --top-fname=aes256_encrypt_ecb --device-name=xc7z020-1clg484-VVD --clock-period=5 -v4 --print-dot |& tee log.txt

## Inspecting the output
After the High-Level Synthesis process has completed successfully, the user can find the generated RTL design in the /content/kernel_offload/aes/aes256_encrypt_ecb.v file.

The HLS tool also generates a synthesis script /content/kernel_offload/aes/synthesize_Synthesis_aes256_encrypt_ecb.sh along with the Verilog description so that it can be used to run the vendor specific synthesis tool to generate the bitstream for the target FPGA.

Furthermore, in the /content/kernel_offload/aes/HLS_output/dot folder you will find many DOT files representing the FSM, State Transition Graph (STG), Control Flow Graph (CFG) of basic block and operations, Control Dependecies Graph (CDG), Operation Dependencies Graph (ODG), and many others. These graphs are useful to understand what kind of transformations and optimizations where performed by the HLS tool and what is the generated accelerator architecture.

In [ ]:
import graphviz
from IPython.display import display

with open("/content/kernel_offload/aes/HLS_output/dot/aes256_encrypt_ecb/HLS_STGraph.dot") as f:
  display(graphviz.Source(f.read()))

# Accelerator Interface Definition
By default, Bambu HLS does not change the interface of the top level function, thus each parameter will be converted into a simple hardware port. Both values and pointers will be passed by value, then memory accesses will be performed through a shared memory bus interface in case it is necessary.

To enable the hardware interface inference based on parameter data type it is necessary to pass the _--generate-interface=INFER_ command line option to the tool. Furthermore, it is possible to explicitly require a given hardware interface through pragmas (for C/C++ inputs) or passing an XML file containing the information through _--architecture-xml=design.xml_.

In [ ]:
%cd /content/kernel_offload/aes
!bambu aes.c -I../common --top-fname=aes256_encrypt_ecb --generate-interface=INFER --device-name=xc7z020-1clg484-VVD --clock-period=5 -v4 --print-dot |& tee log.txt

It is possible to see the new interface by looking at the generated Verilog description /content/kernel_offload/aes/aes256_encrypt_ecb.v

# Verfifying the generated specification
The generated hardware description must be verified through some kind of tests to check compliancy with the original input description. For this purpose, three different methods are available:
 - XML test vectors: _--generate-tb=test.xml_
 - C/C++ testbench: _--generate-tb=aes_test.c_
 - System-level integration: _--generate-tb=elf:aes_

When the _--generate-tb_ command line option is used, the tool will also generate a _simulate_<top_fname>.sh_ bash script which must be called to run the simulation.

The tool may run the simulation automatically after the compilation if the _--simulate_ option is passed through the command line.

## XML Test Vectors
The simplest way to quickly check the generated hardware is to provide some test vectors for the top-level interface through an XML file.

Bambu HLS will feed the test vectors to both the original implementaion and the simulated RTL description and compare the results at bit-level.

_This method is very simple and is not meant to be used for designs where the top-level interface makes use of complex data structures._

In [ ]:
%%writefile /content/kernel_offload/aes/test.aes.xml
<?xml version="1.0"?>
<function>
    <testbench
        ctx="{{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0},{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0},{0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0}}"
        k="{0x00,0x01,0x02,0x03,0x04,0x05,0x06,0x07,0x08,0x09,0x0a,0x0b,0x0c,0x0d,0x0e,0x0f,0x10,0x11,0x12,0x13,0x14,0x15,0x16,0x17,0x18,0x19,0x1a,0x1b,0x1c,0x1d,0x1e,0x1f}"
        buf="{0x00,0x11,0x22,0x33,0x44,0x55,0x66,0x77,0x88,0x99,0xaa,0xbb,0xcc,0xdd,0xee,0xff}" />
</function>

In [ ]:
%cd /content/kernel_offload/aes
!bambu aes.c -I../common --top-fname=aes256_encrypt_ecb --generate-interface=INFER --device-name=xc7z020-1clg484-VVD --clock-period=5 --generate-tb=test.aes.xml --simulate -v4 --print-dot |& tee log.txt

## C/C++ Testbench
A more powerful way to test the generated hardware design is to write a C/C++ program that takes care of the test vectors generation and verification.

The testbench program is allowed to use any kind of function or library, since it is not synthesized into hardware. Dynamic memory allocation, file system access, exceptions, interaction with system I/O devices, passing command line arguments or any other kind of operation which is useful to the user is allowed in the testbench program.

The testbench program source must be provided to the tool using the _--generate-tb=<filename>_ command line option. The option may be used multiple times to specify multiple source files. Additional compilation options may be specified using the _--tb-extra-gcc-options=<flags>_ option.

In this case, since the user-provided testbench may accept command line arguments, the generated _simulate_<top_fname>.sh_ script will act as a wrapper and will forward all arguments to the testbench program.

In [ ]:
%cd /content/kernel_offload/aes
!bambu aes.c -I../common --top-fname=aes256_encrypt_ecb --generate-interface=INFER --device-name=xc7z020-1clg484-VVD --clock-period=5 --generate-tb=aes_test.c --simulate -v4 --print-dot |& tee log.txt

BEAWARE: the default behavior of the tool during simulation is to automatically verify the hardware results against the golden reference generated by executing the input description. To allow this, it is mandatory that the tool knows the exact size of the top level interface parameters.

Parameter size is automatically inferred when the _--generate-interface=INFER_ option is passed, still it is necessary to specify the exact size for pointer parameters whose value is used to access memory through a memory bus (e.g. AXI4 interface), since there is no other way to infer this.

It is possible to specify the size of the memory area pointed by a pointer parameter both statically and dynamically. The static way is to use the _--tb-param-size=<param_name>:<byte_size>_ command line argument. The dynamic way requires to call the _m\_param\_alloc(<param_index>, <byte_size>)_ function every time before the top level function is called.

```
#ifdef __BAMBU_SIM__
#include <mdpi/mdpi_user.h>
#endif
...

int main (int argc, char *argv[])
{
    ...
#ifdef __BAMBU_SIM__
    m_param_alloc(0, sizeof(ctx));
    m_param_alloc(1, sizeof(key));
    m_param_alloc(2, sizeof(buf));
#endif

    aes256_encrypt_ecb(&ctx,key, buf);
    ...
} /* main */
```

It is also possible to disable the autmatic verification performed by the tool by passing the _-DCUSTOM_VERIFICATION_ option through the command line. By doing this, the tool will only take care of the interfacing beteween the testbench and the RTL simulator, without performing any check on the computed results.

## System-level integration
The system-level integration approach allows to integrate the hardware kernel simulation into the host application which will use the kernel. This allows for a full system-level verification of the generated design that will interact with the program directly and without the need to modify the pre-compiled executable.

BEAWARE: The above is possible only if the top level function symbol is a dynamically linked symbol in the pre-compiled executable. Furthermore, to ensure consistency of the shared data structures, it is mandatory that the compiler target is the same for both the kernel and the executable (both must be compiled with -m64 or -m32).

In [ ]:
%cd /content/kernel_offload/aes
!make aes
!bambu aes.c -I../common --top-fname=aes256_encrypt_ecb --generate-interface=INFER --device-name=xc7z020-1clg484-VVD --clock-period=5 --generate-tb=elf:./aes --tb-arg=input.data --tb-arg=check.data --simulate -v4 --print-dot |& tee log.txt

## Memory Access Verification
The Bambu HLS HW/SW Co-Simulation environment also performs validity checks on memory operations, similarly to what happens for software applications when using Valgrind. Since the size of the external memory areas is known, it is possible to exploit the HW/SW API to check for out-of-bounds memory accesses and report them to the user.

By default the simulation enviroment is configured to emulate a Host/Device memory architecture, where the system and accelerator pointers refer to two different memory spaces. This can be changed using the _--tb-memory-mapping=SHARED_ command line option which will enable direct forwarding of pointers to the hardware accelerator to emulate a shared memory architecture where host and accelerator share the same memory.

BEAWARE: Memory checks are NOT performed in shared memory emulation mode, thus erratic memory accesses may result in segmentation fault.